In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, model_from_json
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image
from pathlib import Path
import numpy as np

In [ ]:
# Load the entire data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# List of names for each CIFAR10 class

cifar10_class_names = {
    0: "Plane",
    1: "Car",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Boat",
    9: "Truck"
}

In [ ]:
#import matplotlib.pyplot as plt
# Loop through each picture in the data set
for i in range(1000):
    # Grab an image from the data set
    sample_image = x_train[i]
    # Grab the image's expected class id
    image_class_number = y_train[i][0]
    # Look up the class name from the class id
    image_class_name = cifar10_class_names[image_class_number]

    # Draw the image as a plot
    #plt.imshow(sample_image)
    # Label the image
    #plt.title(image_class_name)
    # Show the plot on the screen
    #plt.show()

In [ ]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
# Our labels are single values from 0 to 9.
# Instead, we want each label to be an array with on element set to 1 and and the rest set to 0.
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(\
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )
model.summary()

In [ ]:
model.fit(
    x_train,\
    y_train,\
    batch_size=32,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True
)

model_structure = model.to_json()
f = Path('model_structure.json')
f.write_text(model_structure)

model.save_weights('model_weights.h5')

In [ ]:
f = Path('model_structure.json')
model_structure = f.read_text()
#model = model_structure.json
model = model_from_json(model_structure)
model.load_weights('model_weights.h5')

In [ ]:
img = image.load_img('cat.png', target_size=(32,32))
image_to_test = image.img_to_array(img) / 255
list_of_images = np.expand_dims(image_to_test, axis=0)
results = model.predict(list_of_images)
single_result = results[0]
most_likely_class_index = int(np.argmax(single_result))
class_likehood = single_result[most_likely_class_index]
class_label = cifar10_class_names[most_likely_class_index]

print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likehood))